# Part 2, Lesson 2: Advanced SQL Features, Window Functions, & R Integration
**Author:** Petr Čala  
**Date:** 2025-03-05

# Part 2 – Lesson 2: Advanced SQL + R

Welcome to the **second 4-hour session** of Part 2! This lesson continues where we left off with **databases and SQL** in R. We’ll focus on:

1. **Recap** & Q&A from Lesson 1
2. **Window Functions** in SQL
3. **Subqueries** & More Complex Joins
4. **Indexing & Performance Tips**
5. **Integrating SQL with R** (dbplyr advanced usage)
6. **Intro to Shiny** for Data Exploration (Optional)
7. **Wrap-Up** & Next Steps

By the end, you’ll be able to write more sophisticated queries and understand how to optimize your database usage in R-driven projects.


---

## 1. Recap & Q&A

In **Part 2, Lesson 1**, you learned:

- When to use a **database** vs. flat files.
- How to **connect** R to an SQLite database (and general references for MySQL/Postgres).
- **Basic SQL** statements (`SELECT`, `INSERT`, `UPDATE`, `JOIN`, `GROUP BY`).
- Simple **ETL** (Extract, Transform, Load) workflow with R + SQLite.

### Check-In

- Do you have any **open questions** or trouble connecting to databases?
- Did you try **loading** your own data into a local SQLite or remote MySQL/Postgres?
- Did you experiment with **joins** and **grouped** queries?

Take a moment to address any issues before diving into advanced topics.


---

## 2. Window Functions in SQL

Window (or **analytic**) functions let you perform calculations **across sets of rows** that are somehow related to the current row. Common examples:

- **Ranking** (RANK, DENSE_RANK, ROW_NUMBER)
- **Running Totals** (SUM(...) OVER ...)
- **Moving Averages**

### 2.1 Window Functions in SQLite

SQLite supports window functions in newer versions (>= 3.25). If using MySQL/Postgres, the syntax is similar (with minor differences). Let’s demonstrate with an **in-memory** SQLite DB.


In [ ]:
library(DBI)
library(RSQLite)
library(dplyr)

# Create or re-use an in-memory SQLite connection
con <- dbConnect(RSQLite::SQLite(), ":memory:")

# Let's make a sample table to demonstrate
df_sales <- data.frame(
  region = c("North", "North", "South", "South", "East", "East", "East"),
  month  = c("Jan", "Feb", "Jan", "Feb", "Jan", "Feb", "Mar"),
  sales  = c(100, 150, 80, 120, 90, 95, 130)
)

dbWriteTable(con, "sales_data", df_sales)
dbListTables(con)


### 2.2 Using a Ranking Window Function

A **ranking** example: Let’s rank each row by `sales` within each `region`.

**SQL** (pseudo-code in SQLite):

```sql
SELECT
    region,
    month,
    sales,
    RANK() OVER (PARTITION BY region ORDER BY sales DESC) AS sales_rank
FROM sales_data;
```

This partitions the data by `region`, then orders by `sales` descending. The `sales_rank` column will show 1 for highest sales, 2 for second highest, etc.


In [ ]:
# Let's run that query in R
res_window <- dbGetQuery(con, "SELECT
                            region,
                            month,
                            sales,
                            RANK() OVER (PARTITION BY region ORDER BY sales DESC) AS sales_rank
                          FROM sales_data;")
res_window


> **Note**: If your SQLite version doesn’t support window functions, you’ll need to upgrade or use a different DB (e.g., Postgres). For MySQL (version 8+), syntax is similar.

### 2.3 Running Totals / Cumulative Sums

Another common use case: **cumulative sums** or **running totals**. Let’s compute a running total of `sales` **by region**, ordered by month.

```sql
SELECT
  region,
  month,
  sales,
  SUM(sales) OVER (
    PARTITION BY region
    ORDER BY month
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS running_total
FROM sales_data;
```

We need the `ORDER BY month` logic to define the time progression. (This example is simplistic because `month` is not numeric, but you get the idea.)


In [ ]:
# Let's just do a quick demonstration with the actual query.
# We'll assume month order is alphabetical for this example.

res_running <- dbGetQuery(con, "SELECT
  region,
  month,
  sales,
  SUM(sales) OVER (
    PARTITION BY region
    ORDER BY month
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS running_total
FROM sales_data;")

res_running


That’s a brief taste of window functions, which can be **extremely** powerful for advanced analysis (rankings, moving averages, etc.).

> **Exercise**: Try using `DENSE_RANK()`, `ROW_NUMBER()`, or computing an **average** over a window.


---

## 3. Subqueries & More Complex Joins

### 3.1 Subqueries

A **subquery** is a query nested inside another. For example, suppose we want to find all employees whose salary is above the **average** salary.

```sql
SELECT *
FROM employees
WHERE salary > (
  SELECT AVG(salary)
  FROM employees
);
```

We can use the same logic in a **WHERE** clause or a **JOIN**. Let’s quickly demonstrate with the `employees` table from the previous lesson (you can re-create it or adapt the concept to your existing data).


In [ ]:
# We'll create a quick employees table in the same in-memory DB to illustrate.

dbExecute(con, "DROP TABLE IF EXISTS employees;")
dbExecute(con, "CREATE TABLE employees (
             emp_id INTEGER PRIMARY KEY,
             name TEXT,
             salary REAL
           );")

dbExecute(con, "INSERT INTO employees (name, salary)
             VALUES ('Alice', 50000),
                    ('Bob',   65000),
                    ('Carla', 40000),
                    ('David', 80000);")

# Subquery to find employees above average salary
above_avg <- dbGetQuery(con, "SELECT name, salary
                         FROM employees
                         WHERE salary > (SELECT AVG(salary) FROM employees);")
above_avg


### 3.2 Complex Joins with Subqueries

We can nest subqueries in `JOIN` conditions or do multi-level subselects. For instance, if you have tables **orders** and **customers**, you might do something like:

```sql
SELECT c.name, c.city, o.order_date, o.amount
FROM customers c
INNER JOIN (
  SELECT *
  FROM orders
  WHERE amount > 100
) AS o
ON c.customer_id = o.customer_id;
```

This filters `orders` to only those above 100 in a **subquery** before joining to `customers`.

> **Exercise**: Incorporate window functions or subqueries into a **join** query on your own sample data.


---

## 4. Indexing & Performance Tips

### 4.1 Why Index?

**Indexes** speed up queries on certain columns by building a data structure (like a B-tree). If you frequently filter by `salary`, indexing `salary` can help. In SQLite:

```sql
CREATE INDEX idx_salary ON employees(salary);
```

When you do `WHERE salary > 50000;`, the DB can quickly locate matching rows.

### 4.2 Query Plans

Databases have **query planners** that decide how to execute queries. For debugging complex queries, you can do:

```sql
EXPLAIN QUERY PLAN SELECT * FROM employees WHERE salary > 50000;
```

This helps you see if the DB is using an index or doing a full table scan.

> **Note**: For large data, also consider normalizing tables, proper **column data types**, and possibly **partitioning** (in bigger DB systems).


---

## 5. Integrating SQL with R More Deeply (dbplyr)

We’ve seen `dbGetQuery()` for raw SQL, but **dbplyr** (part of `dplyr`) lets you write R code that translates to SQL behind the scenes.

### 5.1 Basic dbplyr Setup

```r
library(dbplyr)
tbl_emp <- tbl(con, "employees")   # reference the employees table

tbl_emp %>%
  filter(salary > 50000) %>%
  select(name, salary) %>%
  arrange(desc(salary)) %>%
  show_query()
```

The `show_query()` command reveals the **SQL** that dbplyr generates. When you do a final operation like `collect()`, it pulls the data into R.


In [ ]:
# Let's demonstrate quickly
library(dbplyr)

tbl_emp <- tbl(con, "employees")

tbl_emp %>%
  filter(salary > 50000) %>%
  select(name, salary) %>%
  arrange(desc(salary)) %>%
  show_query()


### 5.2 Using Window Functions via dbplyr

dbplyr also supports some window functions. For instance:

```r
tbl_sales <- tbl(con, "sales_data")

tbl_sales %>%
  group_by(region) %>%
  mutate(rank = min_rank(desc(sales))) %>%
  arrange(region, rank) %>%
  collect()
```

Behind the scenes, dbplyr generates a `RANK()` or `MIN_RANK()` window function call. Keep in mind **not all** window functions are supported in older versions of dbplyr or certain DB engines. But it’s a powerful approach for writing **SQL**-like logic in R syntax.


---

## 6. Intro to Shiny for Data Exploration (Optional)

If time permits, you can show how to build a **Shiny** app to let non-technical users query or visualize data from a database.

### 6.1 Basic Shiny Setup

```r
# install.packages("shiny") if not installed
library(shiny)

ui <- fluidPage(
  titlePanel("Simple DB Query"),
  sidebarLayout(
    sidebarPanel(
      selectInput("region_input", "Region:", choices = c("North", "South", "East"))
    ),
    mainPanel(
      tableOutput("sales_table")
    )
  )
)

server <- function(input, output) {
  output$sales_table <- renderTable({
    query <- paste0("SELECT * FROM sales_data WHERE region = '", input$region_input, "';")
    dbGetQuery(con, query)
  })
}

shinyApp(ui, server)
```

This is a **bare-bones** example. The user can pick a region, and the app queries that subset of `sales_data` in the SQLite database. You can add **plots**, **filters**, or more advanced features.

> **Note**: Shiny runs in the R environment, so ensure your DB connection is accessible. For production, you might use a secure, remote DB.


---

## 7. Wrap-Up & Next Steps

In this **Part 2, Lesson 2**, you covered:

- **Window Functions** (ranking, running totals) for advanced analytics.
- **Subqueries** for more complex logic.
- **Indexing & Performance**: speed up queries with indexes.
- **dbplyr** integration for writing R code that translates to SQL.
- An optional **Shiny** preview to query DB data in a user-friendly interface.

### Where to Go from Here

1. Experiment with **more window function** variants (moving averages, partitioning by multiple columns).
2. Explore real-world data in a multi-table design: practice joins, subqueries, indexing.
3. If relevant, build a **Shiny** app or simple web-based data explorer for your final project.
4. Look into advanced database features: triggers, stored procedures, or advanced query optimization.

### Additional Resources

- **dbplyr** doc: https://db.rstudio.com/dbplyr/
- **Window Functions** doc for your DB engine (e.g., Postgres or SQLite official docs).
- **Shiny** tutorials: https://shiny.rstudio.com/tutorial/
- **Performance** tips: indexing strategies, `EXPLAIN`/`EXPLAIN ANALYZE` usage, data normalization.

This concludes the advanced portion on SQL with R. Continue exploring if you have large datasets or complex relationships that benefit from a relational database approach.

# End of Part 2, Lesson 2
